## Fetch VCF and index

In [13]:
import sgkit
! mkdir -p data
! test ! -e data/chr22.vcf.gz && wget -O data/chr22.vcf.gz http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000G_2504_high_coverage/working/20201028_3202_phased/CCDG_14151_B01_GRM_WGS_2020-08-05_chr22.filtered.shapeit2-duohmm-phased.vcf.gz

In [15]:
! test ! -e data/chr22.vcf.gz.tbi && tabix -f -p vcf data/chr22.vcf.gz

## Fetch ancestral alleles and index

In [16]:
! test ! -e data/ancestral_alleles.tar.gz && wget -O data/ancestral_alleles.tar.gz  ftp://ftp.ensembl.org/pub/release-100/fasta/ancestral_alleles/homo_sapiens_ancestor_GRCh38.tar.gz

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/benj/.wget-hsts'. HSTS will be disabled.
--2023-01-19 00:40:39--  ftp://ftp.ensembl.org/pub/release-100/fasta/ancestral_alleles/homo_sapiens_ancestor_GRCh38.tar.gz
           => ‘data/ancestral_alleles.tar.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-100/fasta/ancestral_alleles ... done.
==> SIZE homo_sapiens_ancestor_GRCh38.tar.gz ... 852605016
==> PASV ... done.    ==> RETR homo_sapiens_ancestor_GRCh38.tar.gz ... done.
Length: 852605016 (813M) (unauthoritative)

homo_sapiens_ancest 100%[===================>] 813.11M  5.30MB/s    in 2m 33s  

2023-01-19 00:43:12 (5.33 MB/s) - ‘data/ancestral_alleles.tar.gz’ saved [852605016]



In [17]:
! cd data && tar -xzf ancestral_alleles.tar.gz

In [18]:
! samtools faidx data/homo_sapiens_ancestor_GRCh38/homo_sapiens_ancestor_22.fa

## Convert VCF to an sgkit dataset

In [1]:
%%time
import sgkit as sg
import xarray as xr
import numpy as np
import dask.array as da
from sgkit.io.vcf import vcf_to_zarr

CPU times: user 1.52 s, sys: 421 ms, total: 1.94 s
Wall time: 2.27 s


In [2]:
%%time
vcf_to_zarr("data/chr22.vcf.gz", "data/chr22.zarr")

CPU times: user 7min 44s, sys: 28.3 s, total: 8min 12s
Wall time: 2min 47s


# Load ancestral states from fasta and save to dataset

In [3]:
import pysam
import sys
fasta = pysam.FastaFile("data/homo_sapiens_ancestor_GRCh38/homo_sapiens_ancestor_22.fa")
# NB! We put in an extra character at the start to convert to 1 based coords.
codec = 'utf-32-le' if sys.byteorder == 'little' else 'utf-32-be'
ancestral_sequence = "X" + fasta.fetch(reference=fasta.references[0])
ancestral_sequence = np.frombuffer(bytearray(ancestral_sequence,codec), dtype="U1")
# From the ancestral states README:
# The convention for the sequence is:
#    ACTG : high-confidence call, ancestral state supported by other 2 sequences
#    actg : low-confidence call, ancestral state supported by one sequence only
#    N    : failure, the ancestral state is not supported by any other sequence
#    -    : the extant species contains an insertion at this position
#    .    : no coverage in the alignment
ds = sg.load_dataset("data/chr22.zarr")
ancestral_states = ancestral_sequence[ds['variant_position'].values]
ancestral_states = xr.DataArray(data=ancestral_states, dims=["variants"], name="variant_ancestral_state")
print("Seen states:")
for val, count in zip(*np.unique(ancestral_states, return_counts=True)):
    print(val, count)
ds.update({"variant_ancestral_state": ancestral_states})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_ancestral_state"}), "data/chr22.zarr", mode="a")

Seen states:
- 51535
. 141300
A 130150
C 252721
G 236782
N 8216
T 127890
a 21735
c 40884
g 37531
t 21657


## Create a mask of sites that have bad ancestral states

In [4]:
ds = sg.load_dataset("data/chr22.zarr")
wanted_variants = da.logical_and(ds['variant_ancestral_state'] != '-',
                     da.logical_and(ds['variant_ancestral_state'] != '.', ds['variant_ancestral_state'] != 'N'))
wanted_variants = wanted_variants.chunk((10000,))
ds.update({"variant_bad_ancestral_mask": xr.DataArray(data=wanted_variants, dims=["variants"], name="variant_bad_ancestral_mask")})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_bad_ancestral_mask"}), "data/chr22.zarr", mode="a")
print(f"{da.sum(~wanted_variants).compute()} sites masked out for bad ancestral state")
assert set(np.unique(ds['variant_ancestral_state'][wanted_variants])) == {'A', 'C', 'G', 'T', 'a', 'c', 'g', 't'}

201051 sites masked out for bad ancestral state


## Create a mask of duplicate positions

In [5]:
ds = sg.load_dataset("data/chr22.zarr")
pos = ds['variant_position']
pos_shift_left = da.full_like(pos,-1)
pos_shift_left[0:-1] = pos[1:]
pos_shift_right = da.full_like(pos,-1)
pos_shift_right[1:] = pos[:-1]
wanted_variants = da.logical_and(pos != pos_shift_left, pos != pos_shift_right)
ds.update({"variant_duplicate_position_mask": xr.DataArray(data=wanted_variants, dims=["variants"], name="variant_duplicate_position_mask")})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_duplicate_position_mask"}), "data/chr22.zarr", mode="a")
print(f"{da.sum(~wanted_variants).compute()} sites masked out for duplicate position")

99504 sites masked out for duplicate position


## Create the combined mask

In [6]:
## Create the combined mask
ds = sg.load_dataset("data/chr22.zarr")
wanted_variants = da.logical_and(ds['variant_duplicate_position_mask'], ds['variant_bad_ancestral_mask'])
ds.update({"variant_mask": xr.DataArray(data=wanted_variants, dims=["variants"], name="variant_mask")})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_mask"}), "data/chr22.zarr", mode="a")
print(f"{da.sum(~wanted_variants).compute()} sites masked out")

276599 sites masked out


In [10]:
ds = sg.load_dataset("data/chr22.zarr")

## Take a subset of the samples for testing

In [12]:
ds = sg.load_dataset("data/chr22.zarr")
wanted_samples = np.zeros((ds.sizes['samples'],), dtype=bool)
wanted_samples[:100] = True
ds = ds.sel(samples=wanted_samples)
sg.save_dataset(ds, "data/chr22.subset.zarr")

/home/benj/projects/ukb/nygc1000_notebook/env/lib/python3.10/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/benj/projects/ukb/nygc1000_notebook/env/lib/python3.10/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


In [1]:
%%time
import tsinfer
import tskit
import numpy as np
sample_data = tsinfer.SgkitSampleData("data/chr22.subset.zarr")
args = {
    "num_threads": 8,
    "progress_monitor": tsinfer.progress.ProgressMonitor(),
}

CPU times: user 701 ms, sys: 706 ms, total: 1.41 s
Wall time: 686 ms


In [3]:
%%time
inf_ts = tsinfer.generate_ancestors(sample_data, path="data/chr22.subset.ancestors", **args)

ga-add   (1/0)100%|██████████| 794k/794k [00:19, 41.4kit/s] 
ga-gen   (2/0)100%|██████████| 60.0k/60.0k [03:47, 264it/s]


CPU times: user 4min 19s, sys: 4.32 s, total: 4min 24s
Wall time: 4min 6s


In [7]:
%%time
ancestors = tsinfer.AncestorData.load("data/chr22.subset.ancestors")
ancestors_ts = tsinfer.match_ancestors(sample_data, ancestors, **args)
ancestors_ts.dump("data/chr22.subset.ancestors.trees")

ma-match (3/0)100%|█████████▉| 59.9k/60.0k [01:16, 627it/s]
ms-muts  (4/0)  0%|          | 0.00/127k [00:00, ?it/s]
ms-muts  (4/0)  0%|          | 1.00/127k [00:00, 3.08it/s]
ms-muts  (4/0)  5%|▍         | 5.89k/127k [00:00, 14.0kit/s]
ms-muts  (4/0)  9%|▉         | 11.9k/127k [00:00, 22.9kit/s]
ms-muts  (4/0) 14%|█▍        | 18.0k/127k [00:00, 29.1kit/s]
ms-muts  (4/0) 19%|█▉        | 24.2k/127k [00:00, 33.7kit/s]
ms-muts  (4/0) 24%|██▍       | 30.4k/127k [00:00, 37.3kit/s]
ms-muts  (4/0) 29%|██▉       | 36.8k/127k [00:00, 40.3kit/s]
ms-muts  (4/0) 34%|███▍      | 43.1k/127k [00:01, 42.6kit/s]
ms-muts  (4/0) 39%|███▉      | 49.4k/127k [00:01, 44.5kit/s]
ms-muts  (4/0) 44%|████▍     | 55.6k/127k [00:01, 46.1kit/s]
ms-muts  (4/0) 49%|████▉     | 61.9k/127k [00:01, 47.4kit/s]
ms-muts  (4/0) 54%|█████▎    | 68.2k/127k [00:01, 48.5kit/s]
ms-muts  (4/0) 58%|█████▊    | 74.2k/127k [00:01, 49.3kit/s]
ms-muts  (4/0) 63%|██████▎   | 80.0k/127k [00:01, 50.0kit/s]
ms-muts  (4/0) 68%|██████▊   | 8

CPU times: user 7min 54s, sys: 4.62 s, total: 7min 59s
Wall time: 1min 20s


In [5]:
%%time
ancestors_ts = tskit.load("data/chr22.subset.ancestors.trees")
tsinfer.match_sample_slice(
                    sample_data,
                    ancestors_ts,
                    indexes=np.arange(
                        0, sample_data.num_samples // 2
                    ),
                    slice_path="data/chr22.subset.ancestors.1.slice",
                    **args
                )

ms-match (1/0)100%|██████████| 100/100 [00:05, 19.9it/s] 

CPU times: user 36.5 s, sys: 237 ms, total: 36.8 s
Wall time: 8.92 s


In [6]:
%%time
ancestors_ts = tskit.load("data/chr22.subset.ancestors.trees")
tsinfer.match_sample_slice(
                    sample_data,
                    ancestors_ts,
                    indexes=np.arange(
                        sample_data.num_samples // 2, sample_data.num_samples
                    ),
                    slice_path="data/chr22.subset.ancestors.2.slice",
                    **args
                )

ms-match (2/0)100%|██████████| 100/100 [00:07, 12.8it/s] 


CPU times: user 40.2 s, sys: 199 ms, total: 40.4 s
Wall time: 11.8 s


In [3]:
%%time
ancestors_ts = tskit.load("data/chr22.subset.ancestors.trees")
inf_ts = tsinfer.combine_sample_slices(
    sample_data,
    ancestors_ts,
    slice_paths=[f"data/chr22.subset.ancestors.1.slice", f"data/chr22.subset.ancestors.2.slice"],
    progress_monitor=tsinfer.progress.ProgressMonitor()
)
inf_ts.dump("data/chr22.subset.inferred.trees")


ms-load  (1/0)100%|██████████| 2.00/2.00 [00:00, 357it/s]A

ms-paths (2/0)  0%|          | 0.00/200 [00:00, ?it/s]
ms-paths (2/0) 25%|██▌       | 50.0/200 [00:00, 498it/s]
ms-paths (2/0)100%|██████████| 200/200 [00:00, 429it/s]

ms-muts  (3/0)  0%|          | 0.00/127k [00:00, ?it/s]
ms-muts  (3/0)  0%|          | 1.00/127k [00:00, 2.88it/s]
ms-muts  (3/0)  2%|▏         | 2.85k/127k [00:00, 6.43kit/s]
ms-muts  (3/0)  4%|▍         | 5.70k/127k [00:00, 10.5kit/s]
ms-muts  (3/0)  7%|▋         | 8.57k/127k [00:00, 13.4kit/s]
ms-muts  (3/0)  9%|▉         | 11.4k/127k [00:00, 15.5kit/s]
ms-muts  (3/0) 11%|█▏        | 14.3k/127k [00:00, 17.1kit/s]
ms-muts  (3/0) 13%|█▎        | 17.1k/127k [00:00, 18.3kit/s]
ms-muts  (3/0) 16%|█▌        | 20.0k/127k [00:01, 19.4kit/s]
ms-muts  (3/0) 18%|█▊        | 22.8k/127k [00:01, 20.2kit/s]
ms-muts  (3/0) 20%|██        | 25.7k/127k [00:01, 20.9kit/s]
ms-muts  (3/0) 22%|██▏       | 28.4k/127k [00:01, 21.4kit/s]
ms-muts  (3/0) 25%|██▍       | 31.2k/127k [00

CPU times: user 49.5 s, sys: 1.45 s, total: 50.9 s
Wall time: 48.8 s


In [4]:
inf_ts.num_sites

793802

In [5]:
inf_ts